# Taller 7: Implementar ejemplos de SimPy 
#### JHONATHAN MEJIA LEON-160003525


## Bank Renege


In [1]:
!pip install simpy

import random
import simpy

In [2]:
RANDOM_SEED = 42 # Semila aleatoria
NEW_CUSTOMERS = 5  # Número total de clientes
INTERVAL_CUSTOMERS = 10.0  # Genera nuevos clientes cada X segundos
MIN_PATIENCE = 3  # Paciencia mínima del cliente
MAX_PATIENCE = 5  # Paciencia máxima del cliente

In [3]:
def source(env, number, interval, counter):
    """
    La fuente genera clientes de manera
    aleatoria.
    """
    for i in range(number):
        c = customer(env, 'Cliente_%02d' % i, counter, time_in_bank=5.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """
    El cliente llega, es atentido y luego se va.
    """
    arrive = env.now
    print('%7.4f %s: Aqui estoy' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Espera al mostrador o se le agota la paciencia
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # El cliente llega al mostrador
            print('%7.4f %s: Esperó %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finalizó' % (env.now, name))

        else:
            # Al cliente se le agotó la paciencia
            print('%7.4f %s: RENEGÓ despues de %6.3f' % (env.now, name, wait))

In [4]:
# Prepara y empieza la simulación
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Empieza los procesos y ejecuta
counter = simpy.Resource(env, capacity=1)
customers_generator = source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter)
env.process(customers_generator)
env.run()

Bank renege
 0.0000 Cliente_00: Aqui estoy
 0.0000 Cliente_00: Esperó  0.000
 1.6081 Cliente_00: Finalizó
10.2006 Cliente_01: Aqui estoy
10.2006 Cliente_01: Esperó  0.000
12.7265 Cliente_02: Aqui estoy
15.8465 Cliente_01: Finalizó
15.8465 Cliente_02: Esperó  3.120
18.5867 Cliente_02: Finalizó
34.9993 Cliente_03: Aqui estoy
34.9993 Cliente_03: Esperó  0.000
35.3018 Cliente_04: Aqui estoy
38.5189 Cliente_03: Finalizó
38.5189 Cliente_04: Esperó  3.217
43.7664 Cliente_04: Finalizó


## Carwash

In [5]:
RANDOM_SEED = 42
NUM_MACHINES = 2  # Número de máquinas en el lavadero
WASHTIME = 5      # Minutos que le toma lavar un carro
T_INTER = 2       # Crea un carro cada ~7 minutos
SIM_TIME = 20     # Tiempo de simulación en minutos

In [6]:
class Carwash(object):
    """
    Un lavadero tiene un número limitado de máquinas NUM_MACHINES
    conas las que puede limpiar carros en paralelo.
    
    Los carros tienen que solicitar una de las máquinas. Cuando
    obtenga una, éstos pueden iniciar el proceso de lavado y esperar
    a que finalice (lo cual toma WASHTIME minutos).
    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        """
        El proceso de lavado. Toma un proceso 'car' e
        intenta limpiarlo.
        """
        yield self.env.timeout(WASHTIME)
        print("El lavadero elimino el %d%% de la suciedad del %s." %
              (random.randint(50, 99), car))


def car(env, name, cw):
    """
    El proceso del carro (cada carro tiene un 'name') llega al
    lavadero ('cw') y solicita una maquina de limpiar.
    
    Luego empieza el proceso de lavado, espera a que finalice y
    se va para no volver nunca más.
    """
    print('%s llega al lavadero a los %.2f minutos.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s entra al lavadero a los %.2f minutos.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s sale del lavadero a los %.2f minutos.' % (name, env.now))


def setup(env, num_machines, washtime, t_inter):
    """
    Crea un lavadero, un número inicial de carros y sigue
    creando carros durante aproximadamente 't_inter' minutos.
    """
    # Crea el lavadero
    carwash = Carwash(env, num_machines, washtime)

    # Crea 4 carros iniciales
    for i in range(4):
        env.process(car(env, 'Carro %d' % i, carwash))

    # Crea más carros mientras la simulación se está ejecutando
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Carro %d' % i, carwash))

In [7]:
# Prepara e inicia la simulación
print('Carwash')
random.seed(RANDOM_SEED)

# Crea un ambiente e inicia el proceso de lavado
env = simpy.Environment()
carwash = setup(env, NUM_MACHINES, WASHTIME, T_INTER)
env.process(carwash)

# Ejecuta!
env.run(until=SIM_TIME)

Carwash
Carro 0 llega al lavadero a los 0.00 minutos.
Carro 1 llega al lavadero a los 0.00 minutos.
Carro 2 llega al lavadero a los 0.00 minutos.
Carro 3 llega al lavadero a los 0.00 minutos.
Carro 4 llega al lavadero a los 0.00 minutos.
Carro 0 entra al lavadero a los 0.00 minutos.
Carro 1 entra al lavadero a los 0.00 minutos.
Carro 5 llega al lavadero a los 0.00 minutos.
Carro 6 llega al lavadero a los 2.00 minutos.
Carro 7 llega al lavadero a los 3.00 minutos.
Carro 8 llega al lavadero a los 4.00 minutos.
El lavadero elimino el 97% de la suciedad del Carro 0.
El lavadero elimino el 56% de la suciedad del Carro 1.
Carro 9 llega al lavadero a los 5.00 minutos.
Carro 0 sale del lavadero a los 5.00 minutos.
Carro 1 sale del lavadero a los 5.00 minutos.
Carro 2 entra al lavadero a los 5.00 minutos.
Carro 3 entra al lavadero a los 5.00 minutos.
Carro 10 llega al lavadero a los 9.00 minutos.
Carro 11 llega al lavadero a los 9.00 minutos.
El lavadero elimino el 77% de la suciedad del Carro 

## Machine Shop


In [8]:
RANDOM_SEED = 42
PT_MEAN = 10.0         # Tiempo promedio de procesamiento en minutos
PT_SIGMA = 2.0         # Sigma del tiempo de procesamiento
MTTF = 300.0           # Tiempo medio para fallar en minutos
BREAK_MEAN = 1 / MTTF  # Parametro para la distribución exponencial
REPAIR_TIME = 30.0     # Tiempo que toma reparar una máquina
JOB_DURATION = 30.0    # Duración de los otros trabajos en minutos
NUM_MACHINES = 10      # Número de máquinas en el taller
WEEKS = 4              # Tiempo de simulación en semanas
SIM_TIME = WEEKS * 7 * 24 * 60  # Tiempo de simulación en minutos

In [9]:
def time_per_part():
    """
    Retorna el tiempo actual de procesamiento para
    una parte en concreto.
    """
    return random.normalvariate(PT_MEAN, PT_SIGMA)


def time_to_failure():
    """
    Retorna el tiempo de la siguiente falla
    en una máquina
    """
    return random.expovariate(BREAK_MEAN)


class Machine(object):
    """
    Una máquina produce partes y se daña de vez en cuando.
    
    Si se rompe, require un reparador y continua la producción luego
    de ser reparada.
    
    Una máquina tiene un nombre y número de partes hechas.
    """
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False

        # Empieza los procesos de 'working' y 'break_machine'
        self.process = env.process(self.working(repairman))
        env.process(self.break_machine())

    def working(self, repairman):
        """
        Produce partes mientras la simulación está corriendo.
        
        Mientras hace una parte, la máquina puede romperse
        multipples veces. Se solicita al reparador cuando esto
        sucede.
        """
        while True:
            # Comienza a hacer una parte
            done_in = time_per_part()
            while done_in:
                try:
                    # Trabajando en la parte
                    start = self.env.now
                    yield self.env.timeout(done_in)
                    done_in = 0  # Se pone en 0 para salir del ciclo.

                except simpy.Interrupt:
                    self.broken = True
                    done_in -= self.env.now - start  # Cuanto tiempo falta?

                    # Solicita un reparador el cual dejara de lado sus otros trabajos
                    with repairman.request(priority=1) as req:
                        yield req
                        yield self.env.timeout(REPAIR_TIME)

                    self.broken = False

            # Part is done.
            self.parts_made += 1

    def break_machine(self):
        """
        Daña la máquina de vez en cuando
        """
        while True:
            yield self.env.timeout(time_to_failure())
            if not self.broken:
                # Solo daña la máquina si actualmente está funcionando.
                self.process.interrupt()


def other_jobs(env, repairman):
    """
    Los otros trabajos no importantes del
    reparador
    """
    while True:
        # Empieza un nuevo trabajo
        done_in = JOB_DURATION
        while done_in:
            # Reintenta el trabajo hasta que esté hecho.
            # La prioridad es más bajo que la de la reparación de las máquinas.
            with repairman.request(priority=2) as req:
                yield req
                try:
                    start = env.now
                    yield env.timeout(done_in)
                    done_in = 0
                except simpy.Interrupt:
                    done_in -= env.now - start

In [10]:
# Prepara y empieza la simulación
print('Machine shop')
random.seed(RANDOM_SEED)

# Crea un ambiente y empiza el proceso de preparación
env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)
machines = [Machine(env, 'Máquina %d' % i, repairman)
            for i in range(NUM_MACHINES)]
env.process(other_jobs(env, repairman))

# Ejecuta!
env.run(until=SIM_TIME)

# Analisis/Resultados
print('Resultados del taller después de %s semanas:' % WEEKS)
for machine in machines:
    print('La %s hizo %d partes.' % (machine.name, machine.parts_made))

Machine shop
Resultados del taller después de 4 semanas:
La Máquina 0 hizo 3251 partes.
La Máquina 1 hizo 3273 partes.
La Máquina 2 hizo 3242 partes.
La Máquina 3 hizo 3343 partes.
La Máquina 4 hizo 3387 partes.
La Máquina 5 hizo 3244 partes.
La Máquina 6 hizo 3269 partes.
La Máquina 7 hizo 3185 partes.
La Máquina 8 hizo 3302 partes.
La Máquina 9 hizo 3279 partes.


## Movie Renege


In [11]:
import collections
RANDOM_SEED = 42
TICKETS = 50  # Número de tiquetes por película
SIM_TIME = 120  # Simular hasta

In [12]:
def moviegoer(env, movie, num_tickets, theater):
    """
    Un cinéfilo intenta compra un número de tiquetes ('num_tickets')
    para una cierta 'movie' en un 'theater'.
    
    Si la pelicula se llena, el cinéfilo deja el teatro. Si el
    cinéfilo llega al mostrador, intenta comprar los tiquetes. Si
    no hay suficiente cantidad de tiquetes, el cinéfilo discute
    y se va.
    
    Si queda menos de un tiquete libre después de que el cinéfilo
    compre sus tiquetes, el evento 'sold out' se dispara causando
    que el resto de cinéfilos se vayan.
    """
    with theater.counter.request() as my_turn:
        # Espera su turno o hasta que la película se lleno
        result = yield my_turn | theater.sold_out[movie]

        # Valida si la pelicula se lleno
        if my_turn not in result:
            theater.num_renegers[movie] += 1
            env.exit()

        # Valida si hay suficientes tiquetes
        if theater.available[movie] < num_tickets:
            # El cinéfilo abandona después de una discusión
            yield env.timeout(0.5)
            env.exit()

        # Compra tiquetes
        theater.available[movie] -= num_tickets
        if theater.available[movie] < 2:
            # Dispara el evento 'sould out'
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(1)


def customer_arrivals(env, theater):
    """
    Crea un nuevo cinéfilo hasta que el tiempo de
    simulación acabe
    """
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))


Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                            'sold_out, when_sold_out, '
                                            'num_renegers')

In [13]:
# Prepara y comienza la simulación
print('Movie Renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Crea el teatro de cine
counter = simpy.Resource(env, capacity=1)
movies = ['El Depredador', 'La Monja', 'Pesadilla Siniestra']
available = {movie: TICKETS for movie in movies}
sold_out = {movie: env.event() for movie in movies}
when_sold_out = {movie: None for movie in movies}
num_renegers = {movie: 0 for movie in movies}
theater = Theater(counter, movies, available, sold_out, when_sold_out,
                  num_renegers)


# Inicia el proceso y ejecuta
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

# Analisis/Resultados
for movie in movies:
    if theater.sold_out[movie]:
        print('La pelicula "%s" se agotó %.1f minutos después de que el mostrador abriera.' % (movie, theater.when_sold_out[movie]))
        print('  Cantidad de personas que se fueron cuando se agotó la pelicula: %s' %
              theater.num_renegers[movie])

Movie Renege
La pelicula "El Depredador" se agotó 38.0 minutos después de que el mostrador abriera.
  Cantidad de personas que se fueron cuando se agotó la pelicula: 16
La pelicula "La Monja" se agotó 43.0 minutos después de que el mostrador abriera.
  Cantidad de personas que se fueron cuando se agotó la pelicula: 5
La pelicula "Pesadilla Siniestra" se agotó 28.0 minutos después de que el mostrador abriera.
  Cantidad de personas que se fueron cuando se agotó la pelicula: 5


## Gas Station Refueling

In [14]:
import itertools

RANDOM_SEED = 42
GAS_STATION_SIZE = 200     # Litros
THRESHOLD = 10             # Umbral para llamar al carrotange (%)
FUEL_TANK_SIZE = 50        # Litros
FUEL_TANK_LEVEL = [5, 25]  # niveles min/max de los carrotanques (litros)
REFUELING_SPEED = 2        # litros por segundo
TANK_TRUCK_TIME = 300      # Segundos que se tarda en llegar el carrotanque
T_INTER = [30, 300]        # Crea un car cada [min, max] segundos
SIM_TIME = 1000            # Tiempo de simulación en segundos

In [15]:
def car(name, env, gas_station, fuel_pump):
    """
    Un carro llega a una estación de gas para repostar.
    
    Este requiere uno de los surtidores de la estación de gas
    e intenta obtener una cantidad deseada de gas de este.
    Si la reserva de la estación está agotada, el carro
    tendrá que esperar hasta que el carrotanque llegue.
    """
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print('%s llegando a la estación de gas a los %.1f segundos' % (name, env.now))
    with gas_station.request() as req:
        start = env.now
        # Solicita uno de los surtidores
        yield req

        # Obtiene la cantidad requerida de combustible
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        yield fuel_pump.get(liters_required)

        # El proceso de repostar toma su tiempo
        yield env.timeout(liters_required / REFUELING_SPEED)

        print('%s terminó de repostar en %.1f segundos.' % (name,
                                                          env.now - start))


def gas_station_control(env, fuel_pump):
    """
    Periodicamente revisa el nivel del 'fuel_pump' y llama al
    carrotanque si el nivel cae por debajo de un umbral.
    """
    while True:
        if fuel_pump.level / fuel_pump.capacity * 100 < THRESHOLD:
            # Necesitamos llamar a un carro tanque ahora!
            print('Llamando el carrotanque a los %.1f segundos' % env.now)
            # Esperamos a que llegue el carro tanque y rellene el deposito
            yield env.process(tank_truck(env, fuel_pump))

        yield env.timeout(10)  # Revisa cada 10 segundos

def tank_truck(env, fuel_pump):
    """
    Llega a la estación de gas después de un cierto retraso
    y la rellena.
    """
    yield env.timeout(TANK_TRUCK_TIME)
    print('Carrotanque llegando a los %.1f segundos' % env.now)
    ammount = fuel_pump.capacity - fuel_pump.level
    print('Carrotanque rellenando %.1f litros.' % ammount)
    yield fuel_pump.put(ammount)


def car_generator(env, gas_station, fuel_pump):
    """
    Genera un nuevo carro que llega a la estación de gas
    """
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('Carro %d' % i, env, gas_station, fuel_pump))

In [16]:
# Prepara e inicia la simulación
print('Gas Station refuelling')
random.seed(RANDOM_SEED)

# Crea el ambiente e inicia los procesos
env = simpy.Environment()
gas_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, GAS_STATION_SIZE, init=GAS_STATION_SIZE)
env.process(gas_station_control(env, fuel_pump))
env.process(car_generator(env, gas_station, fuel_pump))

# Ejecuta!
env.run(until=SIM_TIME)

Gas Station refuelling
Carro 0 llegando a la estación de gas a los 87.0 segundos
Carro 0 terminó de repostar en 18.5 segundos.
Carro 1 llegando a la estación de gas a los 129.0 segundos
Carro 1 terminó de repostar en 19.0 segundos.
Carro 2 llegando a la estación de gas a los 284.0 segundos
Carro 2 terminó de repostar en 21.0 segundos.
Carro 3 llegando a la estación de gas a los 385.0 segundos
Carro 3 terminó de repostar en 13.5 segundos.
Carro 4 llegando a la estación de gas a los 459.0 segundos
Llamando el carrotanque a los 460.0 segundos
Carro 4 terminó de repostar en 22.0 segundos.
Carro 5 llegando a la estación de gas a los 705.0 segundos
Carro 6 llegando a la estación de gas a los 750.0 segundos
Carrotanque llegando a los 760.0 segundos
Carrotanque rellenando 188.0 litros.
Carro 6 terminó de repostar en 29.0 segundos.
Carro 5 terminó de repostar en 76.5 segundos.
Carro 7 llegando a la estación de gas a los 891.0 segundos
Carro 7 terminó de repostar en 13.0 segundos.


## Process Communication


In [17]:
import random

import simpy


RANDOM_SEED = 42
SIM_TIME = 100


class BroadcastPipe(object):
    """A Broadcast pipe that allows one process to send messages to many.

    This construct is useful when message consumers are running at
    different rates than message generators and provides an event
    buffering to the consuming processes.

    The parameters are used to create a new
    :class:`~simpy.resources.store.Store` instance each time
    :meth:`get_output_conn()` is called.

    """
    def __init__(self, env, capacity=simpy.core.Infinity):
        self.env = env
        self.capacity = capacity
        self.pipes = []

    def put(self, value):
        """Broadcast a *value* to all receivers."""
        if not self.pipes:
            raise RuntimeError('There are no output pipes.')
        events = [store.put(value) for store in self.pipes]
        return self.env.all_of(events)  # Condition event for all "events"

    def get_output_conn(self):
        """Get a new output connection for this broadcast pipe.

        The return value is a :class:`~simpy.resources.store.Store`.

        """
        pipe = simpy.Store(self.env, capacity=self.capacity)
        self.pipes.append(pipe)
        return pipe


def message_generator(name, env, out_pipe):
    """A process which randomly generates messages."""
    while True:
        # wait for next transmission
        yield env.timeout(random.randint(6, 10))

        # messages are time stamped to later check if the consumer was
        # late getting them.  Note, using event.triggered to do this may
        # result in failure due to FIFO nature of simulation yields.
        # (i.e. if at the same env.now, message_generator puts a message
        # in the pipe first and then message_consumer gets from pipe,
        # the event.triggered will be True in the other order it will be
        # False
        msg = (env.now, '%s dice Hola en %d' % (name, env.now))
        out_pipe.put(msg)


def message_consumer(name, env, in_pipe):
    """A process which consumes messages."""
    while True:
        # Get event for message pipe
        msg = yield in_pipe.get()

        if msg[0] < env.now:
            # if message was already put into pipe, then
            # message_consumer was late getting to it. Depending on what
            # is being modeled this, may, or may not have some
            # significance
            print('TARDE recibiendo mensaje: en el tiempo %d: %s mensaje recibido: %s' %
                  (env.now, name, msg[1]))

        else:
            # message_consumer is synchronized with message_generator
            print('En el tiempo %d: %s mensaje recibido: %s.' %
                  (env.now, name, msg[1]))

        # Process does some other work, which may result in missing messages
        yield env.timeout(random.randint(4, 8))


# Setup and start the simulation
print('Process communication')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# For one-to-one or many-to-one type pipes, use Store
pipe = simpy.Store(env)
env.process(message_generator('Generador A', env, pipe))
env.process(message_consumer('Consumidor A', env, pipe))

print('\nOne-to-one pipe communication\n')
env.run(until=SIM_TIME)

# For one-to many use BroadcastPipe
# (Note: could also be used for one-to-one,many-to-one or many-to-many)
env = simpy.Environment()
bc_pipe = BroadcastPipe(env)

env.process(message_generator('Generador A', env, bc_pipe))
env.process(message_consumer('Consumidor A', env, bc_pipe.get_output_conn()))
env.process(message_consumer('Consumidor B', env, bc_pipe.get_output_conn()))

print('\nOne-to-many pipe communication\n')
env.run(until=SIM_TIME)

Process communication

One-to-one pipe communication

En el tiempo 6: Consumidor A mensaje recibido: Generador A dice Hola en 6.
En el tiempo 12: Consumidor A mensaje recibido: Generador A dice Hola en 12.
En el tiempo 19: Consumidor A mensaje recibido: Generador A dice Hola en 19.
En el tiempo 26: Consumidor A mensaje recibido: Generador A dice Hola en 26.
En el tiempo 36: Consumidor A mensaje recibido: Generador A dice Hola en 36.
En el tiempo 46: Consumidor A mensaje recibido: Generador A dice Hola en 46.
En el tiempo 52: Consumidor A mensaje recibido: Generador A dice Hola en 52.
En el tiempo 58: Consumidor A mensaje recibido: Generador A dice Hola en 58.
TARDE recibiendo mensaje: en el tiempo 66: Consumidor A mensaje recibido: Generador A dice Hola en 65
En el tiempo 75: Consumidor A mensaje recibido: Generador A dice Hola en 75.
En el tiempo 85: Consumidor A mensaje recibido: Generador A dice Hola en 85.
En el tiempo 95: Consumidor A mensaje recibido: Generador A dice Hola en 95.

## Event Latency

In [18]:
SIM_DURATION = 100


class Cable(object):
    """This class represents the propagation through a cable."""
    def __init__(self, env, delay):
        self.env = env
        self.delay = delay
        self.store = simpy.Store(env)

    def latency(self, value):
        yield self.env.timeout(self.delay)
        self.store.put(value)

    def put(self, value):
        self.env.process(self.latency(value))

    def get(self):
        return self.store.get()


def sender(env, cable):
    """A process which randomly generates messages."""
    while True:
        # wait for next transmission
        yield env.timeout(5)
        cable.put('Sender sent this at %d' % env.now)


def receiver(env, cable):
    """A process which consumes messages."""
    while True:
        # Get event for message pipe
        msg = yield cable.get()
        print('Received this at %d while %s' % (env.now, msg))


# Setup and start the simulation
print('Event Latency')
env = simpy.Environment()

cable = Cable(env, 10)
env.process(sender(env, cable))
env.process(receiver(env, cable))

env.run(until=SIM_DURATION)

Event Latency
Received this at 15 while Sender sent this at 5
Received this at 20 while Sender sent this at 10
Received this at 25 while Sender sent this at 15
Received this at 30 while Sender sent this at 20
Received this at 35 while Sender sent this at 25
Received this at 40 while Sender sent this at 30
Received this at 45 while Sender sent this at 35
Received this at 50 while Sender sent this at 40
Received this at 55 while Sender sent this at 45
Received this at 60 while Sender sent this at 50
Received this at 65 while Sender sent this at 55
Received this at 70 while Sender sent this at 60
Received this at 75 while Sender sent this at 65
Received this at 80 while Sender sent this at 70
Received this at 85 while Sender sent this at 75
Received this at 90 while Sender sent this at 80
Received this at 95 while Sender sent this at 85
